In [2]:
import pandas as pd
import gzip
import json

df = pd.read_csv('./ml-1m/movies.dat', sep='::', names=['movieId', 'title', 'genres'])
df['genres'] = df['genres'].str.split('|')
df = df.explode("genres")
df

<ipython-input-2-e1e0bd37b238>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('./ml-1m/movies.dat', sep='::', names=['movieId', 'title', 'genres'])


,movieId,title,genres
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children's
0,1,Toy Story (1995),Comedy
1,2,Jumanji (1995),Adventure
1,2,Jumanji (1995),Children's
...,...,...,...
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama


In [3]:
item = pd.read_table('./item_list.txt', sep='\s+')
item['org_id']

0          1
1          2
2          3
3          4
4          5
        ... 
3701    3948
3702    3949
3703    3950
3704    3951
3705    3952
Name: org_id, Length: 3706, dtype: int64

In [4]:
cate = pd.read_table('./category_list.txt')
cate['category']

0           Drama
1          Comedy
2          Action
3        Thriller
4         Romance
5          Horror
6       Adventure
7          Sci-Fi
8      Children's
9           Crime
10            War
11    Documentary
12        Musical
13        Mystery
14      Animation
15        Fantasy
16        Western
17      Film-Noir
Name: category, dtype: object

In [5]:
org_id = item.loc[0]['org_id']
meta = df[df['movieId'] == org_id]
meta['genres'].tolist()

['Animation', "Children's", 'Comedy']

In [6]:
import math
import numpy as np
from tqdm import tqdm

i_c = {
    'item_id' : item['remap_id'].tolist(),
    'cate_id' : []
}

for i in tqdm(range(len(i_c['item_id']))):
    org_id = item.loc[i]['org_id']
    meta = df[df['movieId'] == org_id]
    m_cate = meta['genres'].tolist()

    c_id = []
    # print(i)
    '''有两种情况，一种是在meta里没有记录该物品，一种是有记录但类别为NaN'''
    if len(m_cate) == 0:
        i_c['cate_id'].append([18])
        continue
    elif type(m_cate[0]) != str and math.isnan(m_cate[0]):
        i_c['cate_id'].append([18])
        continue
    else:
        for j in range(len(m_cate)):
            # print(cate[cate.category == m_cate[j]]['cate_id'].values)
            c_id.append(int(cate[cate.category == m_cate[j]]['cate_id'].values))
        # print(c_id)
        i_c['cate_id'].append(np.random.choice(c_id))

i_c = pd.DataFrame(i_c)
i_c.to_csv('./item_cate.txt', sep='\t', index=False)
i_c

100%|██████████| 3706/3706 [00:04<00:00, 780.70it/s]


,item_id,cate_id
0,0,1
1,1,6
2,2,1
3,3,0
4,4,1
...,...,...
3701,3701,1
3702,3702,0
3703,3703,0
3704,3704,0
